In [1]:

import pandas as pd
from pommes.io.build_input_dataset import *#build_input_parameters, read_config_file
from pommes.model.data_validation.dataset_check import check_inputs
from pommes.io.save_solution import save_solution
from pommes.model.build_model import build_model
import warnings
import time
from datetime import timedelta
from pathlib import Path
warnings.filterwarnings("ignore")

all_areas=['AL', 'AT', 'BA', 'BE', 'BG', 'CH',
        'CZ', 'DE', 'DK',
         'EE', 'ES', 'FI', 'FR',
         'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU',
         'LV', 'ME', 'MK', 'MT', 'NL', 'NO', 'PL', 'PT',
        'RO', 'RS', 'SE', 'SI', 'SK', 'UK']


# Config


In [ ]:
scenario="GA" #DE or GA
climate_year=2009 #1995 or 2008 or 2009
areas = ["FR", "DE"] #, "BE", "ES", "CH", "IT", "PT", "UK", "IE", "AT", "NL", "DK", "NO", "SE", "PL", "CZ"]
year_op = [2030, 2040, 2050]

Base process

In [2]:

title=f"\033[1m {scenario}CY{climate_year}\033[0m"
print(title)

if areas==["all"]:
    areas=all_areas

add=""
if len(year_op)<4:
    for i in year_op:
        add+="-"+str(i)
suffix = f"{len(areas)}-nodes"+add

print(suffix)
print(year_op,areas)
output_folder = f"output/{scenario}_CY{climate_year}/{suffix}"
start = time.time()
print("\033[1m Input data load and pre-processing \033[0m")
config = read_config_file(study=scenario, file_path="config_pommes.yaml")

###File pathway update
if scenario=="DE":
    config["input"]["path"]="data/Distributed Energy"
elif scenario=="GA":
    config["input"]["path"] = "data/Global Ambition"
else:
    print(404)
    exit()

config["coords"]["area"]["values"]=areas
config["coords"]["year_op"]["values"] = year_op

# config["input"]["parameters"]["demand"]["file"]=f"demand_cy{climate_year}.csv"
config["input"]["parameters"]["conversion_availability"]["file"] = f"availability_cy{climate_year}.csv"
config["input"]["parameters"]["conversion_max_yearly_production"]["file"] = f"conversion_op2_cy{climate_year}.csv"
config["input"]["parameters"]["conversion_power_capacity_max"]["file"] = f"conversion_op2_cy{climate_year}.csv"
config["input"]["parameters"]["conversion_power_capacity_min"]["file"] = f"conversion_op2_cy{climate_year}.csv"

####Links adjustment
if config["add_modules"]["transport"] :
    areas=config["coords"]["area"]["values"]
    all_links=pd.read_csv(config["input"]["path"]+"/transport_link.csv",sep=";").link.unique()
    links = []

    for link in all_links:
        pos = ""
        i = 0
        while pos != "-":
            pos = link[i]
            i += 1
        area_from = link[:i - 1]
        area_to = link[i:]
        if area_to in areas and area_from in areas:
            links.append(link)
    if len(links) >= 1:
        config["coords"]["link"]["values"] = links
    else:
        config["add_modules"]["transport"]=False
print("Transport activated:", config["add_modules"]["transport"])
model_parameters = build_input_parameters(config)
model_parameters = check_inputs(model_parameters)
print("\033[1m Model building \033[0m")
model = build_model(model_parameters)

NameError: name 'scenario' is not defined

## Add multiple weather_year support
